# Inference

In [1]:
# imports
import ast
import os
import pathlib

import neptune
import numpy as np
import torch
from torch.utils.data import DataLoader
from torchvision.models.detection.transform import GeneralizedRCNNTransform

from src.pytorch_faster_rcnn_tutorial.backbone_resnet import ResNetBackbones
from src.pytorch_faster_rcnn_tutorial.datasets import (
    ObjectDetectionDataSet,
    ObjectDetectionDatasetSingle,
)
from src.pytorch_faster_rcnn_tutorial.faster_RCNN import get_faster_rcnn_resnet
from src.pytorch_faster_rcnn_tutorial.transformations import (
    ComposeDouble,
    ComposeSingle,
    FunctionWrapperDouble,
    FunctionWrapperSingle,
    apply_nms,
    apply_score_threshold,
    normalize_01,
)
from src.pytorch_faster_rcnn_tutorial.utils import (
    collate_single,
    get_filenames_of_path,
    save_json,
)
from src.pytorch_faster_rcnn_tutorial.viewers.object_detection_viewer import (
    ObjectDetectionViewer,
    ObjectDetectionViewerSingle,
)
from training_script import NeptuneSettings


In [2]:
# parameters
params = {
    "EXPERIMENT": "HEAD-51",  # experiment name, e.g. Head-42
    "OWNER": "john-judge",  # e.g. johndoe55
    "INPUT_DIR": "src/pytorch_faster_rcnn_tutorial/data/heads/test",  # files to predict
    "PREDICTIONS_PATH": "predictions",  # where to save the predictions
    "MODEL_DIR": "heads",  # load model from checkpoint
    "DOWNLOAD": True,  # whether to download from neptune
    "DOWNLOAD_PATH": "model",  # where to save the model if DOWNLOAD is True
    "PROJECT": "Heads",  # Project name
}


In [3]:
# input files
inputs = get_filenames_of_path(pathlib.Path(params["INPUT_DIR"]))
inputs.sort()

2024-02-27 18:09:40 - INFO - utils.py:27:get_filenames_of_path - Found 5 files in src\pytorch_faster_rcnn_tutorial\data\heads\test


In [4]:
# transformations
transforms = ComposeSingle(
    [
        FunctionWrapperSingle(np.moveaxis, source=-1, destination=0),
        FunctionWrapperSingle(normalize_01),
    ]
)


In [5]:
# create dataset
dataset = ObjectDetectionDatasetSingle(
    inputs=inputs,
    transform=transforms,
    use_cache=False,
)


In [6]:
# create dataloader
dataloader_prediction = DataLoader(
    dataset=dataset,
    batch_size=1,
    shuffle=False,
    num_workers=0,
    collate_fn=collate_single,
)


In [7]:
# environment variables (pydantic BaseSettings class)
neptune_settings: NeptuneSettings = NeptuneSettings()


In [9]:
# import experiment from neptune
project_name = f'{params["OWNER"]}/{params["PROJECT"]}'
project = neptune.init(
    project_qualified_name=project_name, api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiI2Yjg2Y2E2Ny0yMWJkLTRjMjEtODNlNS01NjQwYjNjNDg1MWQifQ=="
)  # get project
experiment_id = params["EXPERIMENT"]  # experiment id
experiment = project.get_experiments(id=experiment_id)[0]
parameters = experiment.get_parameters()
properties = experiment.get_properties()


ProjectNotFound: 

----ProjectNotFound-------------------------------------------------------------------------

Project jjudge3/Heads not found.

Verify if your project's name was not misspelled. You can find proper name after logging into Neptune UI.


In [ ]:
# rcnn transform
transform = GeneralizedRCNNTransform(
    min_size=int(parameters["MIN_SIZE"]),
    max_size=int(parameters["MAX_SIZE"]),
    image_mean=ast.literal_eval(parameters["IMG_MEAN"]),
    image_std=ast.literal_eval(parameters["IMG_STD"]),
)


In [ ]:
# color mapping
color_mapping = {
    1: "red",
}


In [ ]:
# view dataset
datasetviewer = ObjectDetectionViewerSingle(
    dataset=dataset, color_mapping=color_mapping, rcnn_transform=transform
)


In [ ]:
# download model from neptune or load from checkpoint
if params["DOWNLOAD"]:
    download_path = pathlib.Path(os.getcwd()) / params["DOWNLOAD_PATH"]
    download_path.mkdir(parents=True, exist_ok=True)
    model_name = "best_model.pt"  # that's how I called the best model
    # model_name = properties['checkpoint_name']  # logged when called log_model_neptune()
    if not (download_path / model_name).is_file():
        experiment.download_artifact(
            path=model_name, destination_dir=download_path
        )  # download model

    model_state_dict = torch.load(
        download_path / model_name, map_location=torch.device("cpu")
    )
else:
    checkpoint = torch.load(params["MODEL_DIR"], map_location=torch.device("cpu"))
    model_state_dict = checkpoint["hyper_parameters"]["model"].state_dict()


In [ ]:
model = get_faster_rcnn_resnet(
    num_classes=int(parameters["CLASSES"]),
    backbone_name=ResNetBackbones(parameters["BACKBONE"]),  # reverse look-up enum
    anchor_size=ast.literal_eval(parameters["ANCHOR_SIZE"]),
    aspect_ratios=ast.literal_eval(parameters["ASPECT_RATIOS"]),
    fpn=ast.literal_eval(parameters["FPN"]),
    min_size=int(parameters["MIN_SIZE"]),
    max_size=int(parameters["MAX_SIZE"]),
)


/Users/johannes/learnspace/PyTorch-Object-Detection-Faster-RCNN-Tutorial/.venv/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/johannes/learnspace/PyTorch-Object-Detection-Faster-RCNN-Tutorial/.venv/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
# load weights
model.load_state_dict(model_state_dict)

<All keys matched successfully>

In [ ]:
# inference (cpu)
model.eval()
for sample in dataloader_prediction:
    x, x_name = sample
    with torch.no_grad():
        pred = model(x)
        pred = {key: value.numpy() for key, value in pred[0].items()}
        name = pathlib.Path(x_name[0])
        save_dir = pathlib.Path(os.getcwd()) / params["PREDICTIONS_PATH"]
        save_dir.mkdir(parents=True, exist_ok=True)
        pred_list = {
            key: value.tolist() for key, value in pred.items()
        }  # numpy arrays are not serializable -> .tolist()
        save_json(pred_list, path=save_dir / name.with_suffix(".json"))


In [ ]:
# get prediction files
predictions = get_filenames_of_path(
    pathlib.Path(os.getcwd()) / params["PREDICTIONS_PATH"]
)
predictions.sort()


2023-06-08 18:16:23 - INFO - utils.py:27:get_filenames_of_path - Found 8 files in /Users/johannes/learnspace/PyTorch-Object-Detection-Faster-RCNN-Tutorial/predictions


In [ ]:
# create prediction dataset
iou_threshold = 0.25
score_threshold = 0.6

transforms_prediction = ComposeDouble(
    [
        FunctionWrapperDouble(np.moveaxis, source=-1, destination=0),
        FunctionWrapperDouble(normalize_01),
        FunctionWrapperDouble(
            apply_nms, input=False, target=True, iou_threshold=iou_threshold
        ),
        FunctionWrapperDouble(
            apply_score_threshold,
            input=False,
            target=True,
            score_threshold=score_threshold,
        ),
    ]
)

dataset_prediction = ObjectDetectionDataSet(
    inputs=inputs, targets=predictions, transform=transforms_prediction, use_cache=False
)


In [ ]:
# mapping
color_mapping = {
    1: "red",
}

In [ ]:
# visualize predictions
datasetviewer_prediction = ObjectDetectionViewer(
    dataset=dataset_prediction, color_mapping=color_mapping
)


2023-06-08 18:16:25 - INFO - object_detection_viewer.py:42:get_data - Input sample: 001.jpg
Shape: torch.Size([3, 300, 600])
2023-06-08 18:16:25 - INFO - object_detection_viewer.py:64:get_target - Target sample: 001.json
{'boxes': tensor([[334,  72, 425, 210],
        [417,  17, 555, 172],
        [ 53,   2, 168, 115],
        [ 57, 106, 152, 233],
        [186,  61, 253, 164],
        [220,  43, 309, 152]]), 'labels': tensor([1, 1, 1, 1, 1, 1]), 'scores': tensor([0, 0, 0, 0, 0, 0])}
2023-06-08 18:16:26 - INFO - object_detection_viewer.py:42:get_data - Input sample: 002.jpg
Shape: torch.Size([3, 900, 1200])
2023-06-08 18:16:26 - INFO - object_detection_viewer.py:64:get_target - Target sample: 002.json
{'boxes': tensor([[ 148,    0,  411,  301],
        [ 763,   88,  965,  298],
        [ 545,   71,  770,  329],
        [ 572,  329,  848,  670],
        [ 854,  277, 1145,  633],
        [ 214,  370,  604,  900],
        [  36,  439,  267,  754]]), 'labels': tensor([1, 1, 1, 1, 1, 1, 1])

## Experiment with Non-maximum suppression (nms) and score-thresholding

In [ ]:
## currently not available